In [1]:
import tensorflow as tf
import numpy as np
import os
import time
from string import punctuation

In [3]:
text=open('/content/sample_data/text2.txt','rb').read().decode(encoding='utf-8')


In [4]:
vocabularies=sorted(set(text))


In [5]:
vocabularies

[' ',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'إ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٌ',
 'ٍ',
 'َ',
 'ُ',
 'ِ',
 'ّ',
 'ٔ',
 'ٖ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی']

In [6]:
len(vocabularies)

48

In [7]:
cahartoindex={u:i for i,u in enumerate(vocabularies)}
indextochar=np.array(vocabularies)

In [8]:
cahartoindex

{' ': 0,
 'ء': 1,
 'آ': 2,
 'أ': 3,
 'ؤ': 4,
 'إ': 5,
 'ئ': 6,
 'ا': 7,
 'ب': 8,
 'ة': 9,
 'ت': 10,
 'ث': 11,
 'ج': 12,
 'ح': 13,
 'خ': 14,
 'د': 15,
 'ذ': 16,
 'ر': 17,
 'ز': 18,
 'س': 19,
 'ش': 20,
 'ص': 21,
 'ض': 22,
 'ط': 23,
 'ظ': 24,
 'ع': 25,
 'غ': 26,
 'ف': 27,
 'ق': 28,
 'ل': 29,
 'م': 30,
 'ن': 31,
 'ه': 32,
 'و': 33,
 'ٌ': 34,
 'ٍ': 35,
 'َ': 36,
 'ُ': 37,
 'ِ': 38,
 'ّ': 39,
 'ٔ': 40,
 'ٖ': 41,
 'پ': 42,
 'چ': 43,
 'ژ': 44,
 'ک': 45,
 'گ': 46,
 'ی': 47}

In [9]:
text_as_integer=np.array([cahartoindex[c] for c in text])

In [10]:
text_as_integer

array([ 7, 47,  0, ..., 15, 19, 10])

In [11]:
char_dataset= tf.data.Dataset.from_tensor_slices(text_as_integer)

In [12]:
for i in char_dataset.take(10):
    print(indextochar[i.numpy()])

ا
ی
 
ر
س
ت
خ
ی
ز
 


In [13]:
sequences=char_dataset.batch(101,drop_remainder=True)

In [14]:
sequences


<_BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [15]:
for i in sequences.take(4):
    print('--->',''.join(indextochar[i.numpy()]))

---> ای رستخیز ناگهان وی رحمت بی منتهاای آتشی افروخته در بیشه اندیشه هاامروز خندان آمدی مفتاح زندان آمدیبر
--->  مستمندان آمدی چون بخشش و فضل خداخورشید را حاجب تویی اومید را واجب توییمطلب تویی طالب تویی هم منتها ه
---> م مبتدادر سینه ها برخاسته اندیشه را آراستههم خویش حاجت خواسته هم خویشتن کرده رواای روح بخش بی بدل وی 
---> لذت علم و عملباقی بهانه ست و دغل کاین علت آمد وان دواما زان دغل کژبین شده با بی گنه در کین شدهگه مست 


In [16]:
def sit(batch):
    input_text=batch[:-1]
    target_text=batch[1:]
    return input_text, target_text
dataset=sequences.map(sit)


In [17]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [18]:
for i in dataset.take(1):
    print(''.join(indextochar[i[0].numpy()]))
    print(''.join(indextochar[i[1].numpy()]))

ای رستخیز ناگهان وی رحمت بی منتهاای آتشی افروخته در بیشه اندیشه هاامروز خندان آمدی مفتاح زندان آمدیب
ی رستخیز ناگهان وی رحمت بی منتهاای آتشی افروخته در بیشه اندیشه هاامروز خندان آمدی مفتاح زندان آمدیبر


In [19]:
dataset=dataset.batch(64,drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [22]:
vocabulary_size=len(vocabularies)
embedding_dim=256
rnn_units=1024

In [23]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size,embedding_dim),
    tf.keras.layers.GRU(rnn_units,return_sequences=True),
    tf.keras.layers.Dense(vocabulary_size)
])

In [24]:
for input_text,targe_text in dataset.take(1):
    output=model.predict(input_text)
    print(np.shape(output[1]))

2/2 [==============================] - 4s 8ms/step
(100, 48)


In [25]:
si=tf.random.categorical(output[0],num_samples=1)
si

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[26],
       [34],
       [12],
       [33],
       [11],
       [40],
       [22],
       [42],
       [31],
       [22],
       [18],
       [13],
       [15],
       [29],
       [23],
       [35],
       [15],
       [15],
       [18],
       [ 9],
       [22],
       [ 4],
       [ 3],
       [43],
       [44],
       [ 3],
       [19],
       [ 7],
       [35],
       [33],
       [30],
       [11],
       [10],
       [ 0],
       [ 4],
       [42],
       [37],
       [ 2],
       [ 8],
       [ 0],
       [47],
       [32],
       [47],
       [47],
       [41],
       [26],
       [ 0],
       [ 2],
       [ 0],
       [36],
       [46],
       [29],
       [27],
       [29],
       [18],
       [14],
       [34],
       [ 4],
       [10],
       [45],
       [22],
       [32],
       [ 8],
       [16],
       [24],
       [18],
       [32],
       [41],
       [37],
       [33],
       [ 4],
       [13],
       [13],
   

In [26]:
tf.squeeze(si,axis=-1).numpy()

array([26, 34, 12, 33, 11, 40, 22, 42, 31, 22, 18, 13, 15, 29, 23, 35, 15,
       15, 18,  9, 22,  4,  3, 43, 44,  3, 19,  7, 35, 33, 30, 11, 10,  0,
        4, 42, 37,  2,  8,  0, 47, 32, 47, 47, 41, 26,  0,  2,  0, 36, 46,
       29, 27, 29, 18, 14, 34,  4, 10, 45, 22, 32,  8, 16, 24, 18, 32, 41,
       37, 33,  4, 13, 13, 12,  9, 32, 43, 16, 39, 42, 28, 34,  6, 16, 16,
       14, 43, 45, 44,  4, 40, 41, 32, 38,  6, 46,  2, 25,  6, 36])

In [27]:
''.join(indextochar[tf.squeeze(si,axis=-1).numpy()])

'غٌجوثٔضپنضزحدلطٍددزةضؤأچژأساٍومثت ؤپُآب یهییٖغ آ َگلفلزخٌؤتکضهبذظزهُٖوؤححجةهچذّپقٌئذذخچکژؤٖٔهِئگآعئَ'

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         12288     
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 dense_1 (Dense)             (None, None, 48)          49200     
                                                                 
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


In [29]:
def loss_f(labels,logits):
    return(tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True))
model.compile(optimizer='adam', loss=loss_f)

In [30]:
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath='Molana2/checkpoints',save_weights_only=True)

In [31]:
model.fit(dataset,epochs=100,callbacks=[checkpoint])

Epoch 1/100
103/103 [==============================] - 11s 76ms/step - loss: 2.8990
Epoch 2/100
103/103 [==============================] - 6s 54ms/step - loss: 2.4931
Epoch 3/100
103/103 [==============================] - 6s 55ms/step - loss: 2.3988
Epoch 4/100
103/103 [==============================] - 6s 54ms/step - loss: 2.3237
Epoch 5/100
103/103 [==============================] - 6s 55ms/step - loss: 2.2545
Epoch 6/100
103/103 [==============================] - 6s 56ms/step - loss: 2.1848
Epoch 7/100
103/103 [==============================] - 6s 56ms/step - loss: 2.1161
Epoch 8/100
103/103 [==============================] - 6s 56ms/step - loss: 2.0484
Epoch 9/100
103/103 [==============================] - 6s 57ms/step - loss: 1.9805
Epoch 10/100
103/103 [==============================] - 6s 56ms/step - loss: 1.9096
Epoch 11/100
103/103 [==============================] - 6s 58ms/step - loss: 1.8314
Epoch 12/100
103/103 [==============================] - 6s 61ms/step - loss: 1.7395


In [32]:
tf.train.latest_checkpoint('')

In [33]:
model_2=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size,embedding_dim),
    tf.keras.layers.GRU(1024,return_sequences=True),
    tf.keras.layers.Dense(vocabulary_size)
])

In [34]:
model_2.load_weights(tf.train.latest_checkpoint('/content/Molana2/'))

In [35]:
model_2.build(tf.TensorShape([1,None]))

In [41]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         12288     
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 dense_2 (Dense)             (None, None, 48)          49200     
                                                                 
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


In [43]:
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [cahartoindex[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)
input_eval

<tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[ 8, 32,  0, 31,  7, 30,  0, 14, 15,  7, 33, 31, 15,  0, 12,  7,
        31,  0, 33,  0, 14, 17, 15]], dtype=int32)>

In [46]:
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    predictions = model_2.predict(input_eval)
    predictions = tf.squeeze(predictions, 0)
#     predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy()
    predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
    print(predicted_ids)
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    input_eval = tf.expand_dims(message, 0)
    text_generated.append(indextochar[predicted_ids])




1/1 [==============================] - 0s 21ms/step
47
1/1 [==============================] - 0s 20ms/step
7
1/1 [==============================] - 0s 21ms/step
29
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 23ms/step
12
1/1 [==============================] - 0s 23ms/step
8
1/1 [==============================] - 0s 18ms/step
7
1/1 [==============================] - 0s 20ms/step
8
1/1 [==============================] - 0s 20ms/step
7
1/1 [==============================] - 0s 23ms/step
47
1/1 [==============================] - 0s 19ms/step
15
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 25ms/step
8
1/1 [==============================] - 0s 21ms/step
47
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 21ms/step
10
1/1 [==============================] - 0s 21ms/step
33
1/1 [==============================] - 0s 23ms/step
0
1/1 [===============

In [48]:
''.join(text_generated).split('\n')

['به نام خداوند جان و خردیال جباباید بی تو شد جملگ تو را چون مذرب باشدوان راکه من و مست همی کردم تیزماندر دل میدان شده دستار ما راپیش ز نورش به روشن از این همه گفتار راچون در افلاک دگر باشد در جوی جانتا در آن سحرت کی رود شوخ هر شبی بی نماید روانیخواب چو راه ده قدح با تو درآید در جوش این دیگ جانستچون رسید آن جا که یاریست به دلت رازندگان را گرو نام شد از آن مه که بهار آید ز دوغ عمر باشد درخور از آن رنگ بدیدیتا لذاش تو نبود جوشش و روح جمله شاهان در حدست و عنقاست ندامستجان را تو که عاری وا لاکمال فیه از تو آن همه اوصاف خویش']

In [50]:
for i in ''.join(text_generated).split('\n'):
    print(i)

به نام خداوند جان و خردیال جباباید بی تو شد جملگ تو را چون مذرب باشدوان راکه من و مست همی کردم تیزماندر دل میدان شده دستار ما راپیش ز نورش به روشن از این همه گفتار راچون در افلاک دگر باشد در جوی جانتا در آن سحرت کی رود شوخ هر شبی بی نماید روانیخواب چو راه ده قدح با تو درآید در جوش این دیگ جانستچون رسید آن جا که یاریست به دلت رازندگان را گرو نام شد از آن مه که بهار آید ز دوغ عمر باشد درخور از آن رنگ بدیدیتا لذاش تو نبود جوشش و روح جمله شاهان در حدست و عنقاست ندامستجان را تو که عاری وا لاکمال فیه از تو آن همه اوصاف خویش
